In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

# Load MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Define a Multi-Layer Perceptron (MLP) model
class MLP(nn.Module):
    def __init__(self, hidden_layers):
        super(MLP, self).__init__()
        layers = [nn.Linear(28*28, 128), nn.ReLU()]  # Input layer
        for _ in range(hidden_layers):
            layers.append(nn.Linear(128, 128))  # Hidden layer
            layers.append(nn.ReLU())
        layers.append(nn.Linear(128, 10))  # Output layer
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten 28x28 images to 784
        return self.model(x)

# Train and evaluate function
def train_and_evaluate(hidden_layers, epochs=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = MLP(hidden_layers).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    print(f"\nTraining model with {hidden_layers} hidden layers...")

    # Training loop
    for epoch in range(1, epochs + 1):  
        running_loss = 0.0
        correct, total = 0, 0

        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Compute accuracy on training data
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        avg_loss = running_loss / len(trainloader)  
        train_accuracy = 100 * correct / total

        print(f"Epoch {epoch}: Loss: {avg_loss:.4f}, Accuracy: {train_accuracy:.2f}%")

    # Testing phase
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = 100 * correct / total
    print(f"Test Accuracy: {test_accuracy:.2f}%\n")

# Train models with different hidden layers
for layers in [3, 5, 10]:
    train_and_evaluate(layers, epochs=5)



Training model with 3 hidden layers...
Epoch 1: Loss: 0.4029, Accuracy: 87.16%
Epoch 2: Loss: 0.1755, Accuracy: 94.59%
Epoch 3: Loss: 0.1354, Accuracy: 95.84%
Epoch 4: Loss: 0.1136, Accuracy: 96.48%
Epoch 5: Loss: 0.0982, Accuracy: 96.87%
Test Accuracy: 96.51%


Training model with 5 hidden layers...
Epoch 1: Loss: 0.4703, Accuracy: 84.46%
Epoch 2: Loss: 0.1923, Accuracy: 94.10%
Epoch 3: Loss: 0.1439, Accuracy: 95.69%
Epoch 4: Loss: 0.1224, Accuracy: 96.39%
Epoch 5: Loss: 0.1078, Accuracy: 96.80%
Test Accuracy: 96.19%


Training model with 10 hidden layers...
Epoch 1: Loss: 0.8368, Accuracy: 70.33%
Epoch 2: Loss: 0.3346, Accuracy: 91.17%
Epoch 3: Loss: 0.2816, Accuracy: 92.61%
Epoch 4: Loss: 0.2567, Accuracy: 93.44%
Epoch 5: Loss: 0.2098, Accuracy: 94.61%
Test Accuracy: 95.50%

